In [3]:
import shutil
from pathlib import Path
# test will only work on first importt of ea
# remove cache folder and then import ea to load it again
cache_path = Path('./cache')
shutil.rmtree(cache_path, ignore_errors=True)

# path should be removed
assert cache_path.is_dir() == False

# import ea will load the new folder
import equilibrator_a as ea
assert cache_path.is_dir() == True
assert Path('./cache/compounds.sqlite').is_file()

Local compounds.sqlite not found. Exporting from default equilibrator/cache quilt repo.
If another quilt repo is to be used, run generate_new_ccache with specified repo.
/Users/kevbot/Library/Application Support/QuiltCli/quilt_packages
equilibrator/cache             latest               d11d9ebd1cf25d95d8c1369a8ca2bbe46eb734f8b2b9a4225f16de614104e12c
equilibrator/component_contribution                      2c14d76366ba17dc876f2fc10612db41cae65c0411d8a654b762faa2beb3b055
equilibrator/component_contribution latest               fd23b10c939089ffe0409eac32e36f36ab2fa3d3e486f008acde963ffd5a2ad8
kevbot/cache_mod               latest               3fcbdd88efa0025916e90ecd857fdb88a000567690fbdeffbeabb416a5684c1f
kevbot/test                    latest               db691ff8f3a1ff5af03f988cc84f29cef8f188d7dbc9dcbed892822f2f906d69
equilibrator/cache already installed.
Fragments already downloaded
Exporting file 2 of 2 [00:03]: compounds.sqlite


# Ensuring compounds are saved into the database at a local level 
The first call uses cxcalc, so will take longer. The second time the compound is added to the database and won't take as long.

Second run through the compound should already exist when run.

In [1]:
import equilibrator_a as ea

Fragments already downloaded


In [2]:
import time
# First query is long, because cxcalc, second is short, because already exists in DB
mol_string = 'OC(=O)CCCCOCCO'
# mol_string = 'CCCCCO'
# inchi_partial = 'GTIFLSYPOXYYFG'
# rest = cc.search_compound_by_inchi_key(inchi_partial)
# if rest: 
#     print('Already exists')
# else:
#     print('Doesn\'t exist, Generating compound')

then = time.time()
cpd_get = ea.get_compound(mol_string)
print(f'ID on first query is: {cpd_get.id}: {time.time() - then} s')
then = time.time()
cpd_get = ea.get_compound(mol_string)
print(f'ID on second query is: {cpd_get.id}: {time.time() - then} s')

found match
ID on first query is: 694325: 0.41056108474731445 s
found match
ID on second query is: 694325: 0.39617466926574707 s


In [8]:
ea.commit_ccache()

In [3]:
from component_contribution.predict import GibbsEnergyPredictor
# _gen_compound doesn't assign an id
cpd_gen = ea._gen_compound(mol_string)
cpd_gen.id = -1
GP = GibbsEnergyPredictor()
print(GP.standard_dgf(cpd_get))
print(GP.standard_dgf(cpd_gen))

Fragments already downloaded
(-595.6 +/- 2.2) kilojoule / mole
(-595.6 +/- 2.2) kilojoule / mole


In [5]:
cpd_gen.id

In [6]:
from equilibrator_cache import Compound
import equilibrator_a as ea
cc = ea._get_ccache()
compound_id = 694325
cpd_q = cc.session.query(Compound).filter_by(id=694325).one_or_none()
cpd_q.mass

AttributeError: 'list' object has no attribute 'strip'

# Stress testing multiple smiles in a row

In [5]:
import equilibrator_a as ea
import time
mol_strings = ['OC(=O)CCOCCO', 'OC(=O)CCCCCOCCO', 'OC(=O)CCCCCCCOCCO', 
               'OC(=O)CCCCCCCCOCCO', 'OC(=O)CCCCCCCCOCCO', 'OC(=O)COCCCCOCCO', 
               'OC(=O)CCCOCCCOCCO', 'OC(=O)CCCCCOCCOCCO']

then = time.time()
cpds = [ea.get_compound(mol_string) for mol_string in mol_strings]
print(f'Elapsed time: {time.time() - then}')

found match
found match
found match
found match
found match
found match
found match
found match
Elapsed time: 3.204732894897461


In [3]:
from component_contribution.predict import GibbsEnergyPredictor
GP = GibbsEnergyPredictor()

Fragments already downloaded


In [4]:
dg = [GP.standard_dgf(cpd) for cpd in cpds]

In [5]:
dg

[<Measurement(-609.9390964436857, 1.880140310205774, kilojoule / mole)>,
 <Measurement(-588.4753659921429, 2.4216596613313732, kilojoule / mole)>,
 <Measurement(-574.166212357781, 2.987862661274217, kilojoule / mole)>,
 <Measurement(-567.0116355406001, 3.3032910726483844, kilojoule / mole)>,
 <Measurement(-567.0116355406001, 3.3032910726483844, kilojoule / mole)>,
 <Measurement(-706.7846020989515, 3.486893692815839, kilojoule / mole)>,
 <Measurement(-699.6300252817705, 3.626228198820541, kilojoule / mole)>,
 <Measurement(-692.4754484645896, 3.800343506983618, kilojoule / mole)>]